In [1]:
import json

In [2]:
f = open('mpqa_csds.json', 'r', encoding='utf-8')

In [3]:
data = json.load(f)

In [4]:
import re

def replace_punc(clean_text, offset, offset_list):
    poss_diff = 0
    s = clean_text
    s, off1 = re.subn(' \.', '.', s)
    s, off2 = re.subn(' ,', ',', s)
    s, off3 = re.subn(' ;', ';', s)
    s, off4 = re.subn(' :', ':', s)
    s, off5 = re.subn(' POSS', '\'s', s)
    if off5 == 1:
        poss_diff = 3
    adjust_val = off1 + off2 + off3 + off4 + poss_diff
    offset -= adjust_val
    if (off1 + off2 + off3 + off4 + poss_diff > 0):
        offset_list[-1] -= adjust_val
    return s, offset, offset_list

In [5]:
def quote_helper(offset, offset_list, quote_bool, index, clean_text, w_text):
    if not quote_bool:
        clean_text += ' ' + w_text[index] + w_text[index+1]
        offset += 1
        offset_list.append(offset)
        offset += len(w_text[index])
        offset_list.append(offset)

        #for the word itself
        index+=1
        offset += len(w_text[index])
        quote_bool = True
    else:
        clean_text += w_text[index]
        offset_list.append(offset)
        offset += 1
        quote_bool = False
    return offset, offset_list, quote_bool, index, clean_text, w_text

In [6]:
def assemble_tokens(w_text):
    clean_text = ''
    offset = 0
    offset_list = [0]
    index = 0
    in_double_quote = False
    in_single_quote = False
    while index < len(w_text):
        if (index == 0):
            #if first index is quote:
            if w_text[index] == '"':
                clean_text += w_text[index] + w_text[index+1]
                offset_list.append(offset)
                offset += len(w_text[index])
                offset_list.append(offset)

                #for the word itself
                index+=1
                offset += len(w_text[index])
                in_double_quote = True
            else:
                clean_text += w_text[index]
                offset += len(w_text[index])
        elif w_text[index] == '"':
            offset, offset_list, in_double_quote, index, clean_text, w_text = quote_helper(offset, offset_list, in_double_quote, index, clean_text, w_text)
        elif w_text[index] == '\'':
            offset, offset_list, in_single_quote, index, clean_text, w_text = quote_helper(offset, offset_list, in_single_quote, index, clean_text, w_text)
        else:
            clean_text += ' '
            offset += 1
            offset_list.append(offset)
            clean_text += w_text[index]    
            offset += len(w_text[index])
        clean_text, offset, offset_list = replace_punc(clean_text, offset, offset_list)
        index += 1
    return clean_text, offset_list

In [41]:
def testing(limit):
    count = 0
    punc_list = ['.', ',', ';', ':', '\"', '\'']
    for obj in data:
        if count > limit:
            break
        s1 = set(['w_head_span', 'w_text', 'clean_head', 'head_start', 'head_end'])
        w_text = obj['w_text']
        for ind in range(len(obj['w_text'])):
            if w_text[ind] == '\'s':
                w_text[ind] = "POSS"
        if s1.issubset(obj.keys()):
            clean, offset_list = assemble_tokens(w_text)
            w_head, w_end = obj['w_head_span']
            # print(clean)
            print('ORIGINAL:', obj['clean_head'])
            while w_end >= len(offset_list):
                w_end -= 1
            s_head = offset_list[w_head]
            e_head = offset_list[w_end]
            if w_head == 0 and w_end == 0:
                print('     NEW:', '')
            #deal with punctuation since they won't have spaces before them (commas will be two spaces away, quotes one space away)
            elif offset_list[-1] == e_head or clean[offset_list[w_end+1]-2] in punc_list or clean[offset_list[w_end+1]-1] in punc_list:
                #if whole sentence is the head
                if w_text[w_end] == '.':
                    print(' a   NEW:', clean[s_head:])
                else:
                    print(' a   NEW:', clean[s_head:e_head])
            else:
                print('     NEW:', clean[s_head:e_head-1])
        count+=1

In [42]:
testing(500)

ORIGINAL: branded as "evil "
     NEW: branded as "evil"
ORIGINAL: "nuclear club "
 a   NEW: "nuclear club".
ORIGINAL: may have erred
 a   NEW: may have erred.
ORIGINAL: more clearly
     NEW: more clearly
ORIGINAL: real
     NEW: real
ORIGINAL: as far as human rights are concerned
 a   NEW: as far as human rights are concerned
ORIGINAL: easily qualify as repressive and violating human rights
 a   NEW: easily qualify as repressive and violating human rights.
ORIGINAL: oppressive
 a   NEW: oppressive.
ORIGINAL: not in itself
     NEW: not in itself
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: evildoer
 a   NEW: evildoer.
ORIGINAL: therefore
     NEW: therefore
ORIGINAL: scores of
     NEW: scores of
ORIGINAL: seemingly
     NEW: seemingly
ORIGINAL: Perhaps
     NEW: Perhaps
ORIGINAL: In other words
 a   NEW: In other words
ORIGINAL: aggressively
     NEW: aggressively
ORIGINAL: grave human rights violators
 a   NEW: grave human rights violators
ORIGINAL: irrespective
     NEW

IndexError: list index out of range

In [ ]:
w_text[28]

In [ ]:
len(w_text)

In [ ]:
w_head_span = [0, 29]

w_text = [
                "Come",
                "to",
                "think",
                "of",
                "it",
                ",",
                "acquiring",
                "weapons",
                "of",
                "mass",
                "destruction",
                "per",
                "se",
                "may",
                "not",
                "be",
                "a",
                "correct",
                "basis",
                "for",
                "judging",
                "whether",
                "a",
                "nation",
                "is",
                "good",
                "or",
                "evil",
                "."
            ]

In [ ]:
w_text = [ "\"",
                "PRESIDENT",
                "GEORGE",
                "Bush",
                "'s",
                "National",
                "Security",
                "Adviser",
                "Condoleezza",
                "Rice",
                "has",
                "recently",
                "\'",
                "defined",
                "\'",
                "the",
                "context",
                "and",
                "scope",
                "of",
                "the",
                "term",
                "\'",
                "axis",
                "of",
                "evil",
                "\'",
                "that",
                "Bush",
                "used",
                "in",
                "his",
                "State",
                "of",
                "the",
                "Union",
                "address",
                "a",
                "few",
                "weeks",
                "ago",
                "to",
                "describe",
                "Iraq",
                ",",
                "Iran",
                "and",
                "North",
                "Korea",
                ".", "\""]

In [ ]:
for ind in range(len(w_text)):
    if w_text[ind] == '\'s':
        w_text[ind] = "POSS"
clean, off_list = assemble_tokens(w_text)
clean